<a target="_blank" href="https://colab.research.google.com/github/farahshamout/AIP-2024/blob/main/Week%204/%5BDay%202%2C%20Task%202%5D%20Chatbot%20Architectures.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Retrieval Augmented Generation and Chatbot Architectures

In this notebook we will work through what it takes to build a chatbot from the most basic version, all the way through to a more complicated chatbot that uses a conversational agent with tools as well as guardrails.

In [4]:
import os
os.environ["OPENAI_API_KEY"] = "sk-..."  #put the API key in here

In [7]:
%pip install --upgrade gradio
%pip install langchain
%pip install langchain-community
%pip install openai
%pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.8 MB/s eta 0:00:00


## The most basic Chatbot possible:

This simply uses a user interface attached to a LLM

In [8]:
from langchain_openai import ChatOpenAI
import gradio as gr

llm = ChatOpenAI(temperature=1.0, model='gpt-3.5-turbo-16k')

def predict(message, history):
    gpt_response = llm.invoke(message)
    return gpt_response.content

gr.ChatInterface(predict).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2724d60175b596b0c6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## Retrieval Augmented Generation Example

### Source and clean data for RAG

In [ ]:
# %pip install bs4

In [9]:
import requests
from bs4 import BeautifulSoup
import re

def scrape_website_text(url):
    # Send a GET request to the Wikipedia page
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract all the text from paragraphs and other relevant tags
        paragraphs = soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
        text = ' '.join([para.get_text() for para in paragraphs])

        # Clean the text by removing special characters
        cleaned_text = re.sub(r'[^A-Za-z0-9\s]', '', text)

        return cleaned_text.strip()
    else:
        return "Failed to retrieve the page"

In [10]:
url = """https://www.cbsnews.com/news/paris-olympics-2024-200-meters-noah-lyles-kenny-bednarek-letsile-tebogo/"""  # put any website page in here
cleaned_text = scrape_website_text(url)

# you can test that this has worked using the line below (uncomment to use)
# print(cleaned_text)

### Split and chunk the text for embedding

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

In [12]:
texts = text_splitter.create_documents([cleaned_text])
print(texts[0])
print(texts[1])

page_content='Watch CBS News Noah Lyles of Team USA wins bronze in 200 meters after testing positive for COVID Botswanas Letsile Tebogo claims gold 
    By
                        
                      Allison Elyse Gualtieri
 
Updated on  August 8 2024  819 PM EDT
           CBS News'
page_content='Noah Lyles sought to follow up his gold medal in the 100 meters with a matching one in the 200 meters On Thursday the favorite instead claimed the bronze in the 2024 Paris Olympics  and revealed that he had been diagnosed with COVID19 two days earlier  He finished behind Letsile Tebogo of Botswana who won gold and fellow American Kenny Bednarek who garnered the silver  Lyles a 27yearold from northern Virginia said in an interview with NBC after the race that he woke up early Tuesday feeling really horrible  I knew it was more than just being sore from the 100 he said Woke up the doctors and we tested and unfortunately it came up that I was positive for COVID  Lyles was seen almost immediat

### Embed the documents and save in a Vector database

In [15]:
%pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 40.9 MB/s eta 0:00:00


The Cell below does cost money to run, however the embeddings don't have to be done every time. Once a vector database is made, it can be saved and retrieved for later use.

In [17]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

vector_database = FAISS.from_documents(texts, embedding_model)

Below is a test to see if we can retrieve some relevant context chunks from our document store:

In [18]:
query = "Who won gold in the men's 200m sprint at the 2024 Paris Olympics?"

print(vector_database.similarity_search(query, 3))

[Document(page_content='by a mere five thousandths of a second American Fred Kerley rounded out the medals in photo finish with the top four finishers separated by just 04 seconds and the top seven just 09  a literal blink of an eye Bednarek claimed the silver in the 200 meters in Tokyo but fell behind in the 100 meters in Paris finishing seventh at the Stade de France Lyles also took home the bronze in the 200 meters in Tokyo He added the 100meter sprint for the Paris Games which paid off earlier this week The 2024 Summer Games have been good for Team USA which led the medal count in athletics  the track and field events  going into competition Friday six gold seven silver and six bronze\xa0 2024 Summer Olympics in Paris Allison Elyse Gualtieri is a Senior News Editor for CBSNewscom working on a wide variety of subjects including crime longerform features and feelgood news She previously worked for the Washington Examiner and US News and World Report among other outlets'), Document(pa

### Include the RAG pipeline into our chatbot prototype:

In [19]:
from langchain_openai import ChatOpenAI
import gradio as gr

llm = ChatOpenAI(temperature=1.0, model='gpt-3.5-turbo-16k')

def predict(message, history):
    context = vector_database.similarity_search(message, 3)
    formatted_prompt = f"""{context} \nUse the above context to answer the follwing question: \n{message}"""
    gpt_response = llm.invoke(formatted_prompt)
    return gpt_response.content

gr.ChatInterface(predict).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://37d575e90377491040.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## RAG Chatbot with Conversational History

In [20]:
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import gradio as gr

llm = ChatOpenAI(temperature=1.0, model='gpt-3.5-turbo-16k')

def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))
    context = vector_database.similarity_search(message, 3)
    formatted_prompt = f"""{context} \nUse the above context to answer the follwing question: \nHere is your conversation history with the user and their latest question: {history_langchain_format}"""
    print(history_langchain_format)
    print(context)
    gpt_response = llm.invoke(formatted_prompt)
    return gpt_response.content

gr.ChatInterface(predict).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ec889ea76830a462cd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## Conversational Agent

### import the libraries

In [21]:
from langchain.agents import Tool
from langchain.tools import tool
from langchain.globals import set_debug
set_debug(True)

In [35]:
%pip install wikipedia
# %pip install langchainhub

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=eee7759ea69c2e21587f203aabfd5798d3a7dde379978989ac19e42a89543876
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


### Define the tools

In [23]:
# defining the tool for the context retrieval function
@tool
def context_retreival(query: str) -> str:
    """This tool returns relevant context documents about the 200m sprint at the 2024 olympics in Paris. Ask this tool any question about the 2024 200m sprint finals and who won it."""

    context = vector_database.similarity_search(query, 3)

    return context

In [36]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# defining the tool for the wiki retrieval function
@tool
def wiki_lookup(query: str) -> str:
    """This tool should be used for questions about particular people and their achievements. It returns a context document from wikipedia."""

    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

    context = wikipedia.run(query)

    return context

In [25]:
tools = [
    context_retreival,
    wiki_lookup
]

### Initialise the prompt and agent

In [28]:
# from langchain import hub
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# The commented out lines below did not work in colab, so I have manually recreated the prompt in the lines below those
# Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/openai-functions-agent")

assistant_system_message = """You are a helpful assistant. \
Use tools (only if necessary) to best answer the users questions."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", assistant_system_message),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [29]:
from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [30]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

### Launch a user interface for experimentation:

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import gradio as gr

llm = ChatOpenAI(temperature=1.0, model='gpt-3.5-turbo-16k')

def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))

    gpt_response = agent_executor.invoke({"input": message, "chat_history": history_langchain_format})
    return gpt_response['output']


gr.ChatInterface(predict).launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://6cd8430b98b8d47a3d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[chain/start] [chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": []
}
[chain/end] [chain:AgentExecutor 